In [ ]:
"""
Quick and Compact Bezier Curve Detection with Fast Meta CNNs
"""

import glob
import pickle
import os
import sys
import random

import numpy as np
import tensorflow as tf

import tensorpack as tp
from tensorpack.tfutils.summary import add_moving_summary
from tensorpack.tfutils.scope_utils import auto_reuse_variable_scope
import tensorpack.tfutils.symbolic_functions as symbf

In [ ]:
DIMS = (64, 64)
BATCH = 100
EX_PER_FILE = 100

NET_OPTION = 'fwb'
USE_IDYX = True

In [ ]:
SAVE_DIR = './data/'
MASK_DIR = os.path.join(SAVE_DIR, 'masks')
PARAM_DIR = os.path.join(SAVE_DIR, 'params')
WIDTH_DIR = os.path.join(SAVE_DIR, 'widths')

# Special dataflow
class LineDataFlow(tp.RNGDataFlow):
    
    def __init__(self, filenames, do_shuffle):
        self.filenames = filenames
        self.do_shuffle = do_shuffle
        
    def size(self):
        return len(self.filenames) * EX_PER_FILE
    
    def get_data(self):
        for name in self.filenames:
            masks = np.load(os.path.join(MASK_DIR, name))
            params = np.load(os.path.join(PARAM_DIR, name))
            widths = np.load(os.path.join(WIDTH_DIR, name))
            
            idx = np.r_[:masks.shape[0]]
            if self.do_shuffle:
                np.random.shuffle(idx)
            for i in idx:
                yield [masks[i], params[i], widths[i]]

In [ ]:
all_files = ['{:02}.npy'.format(i) for i in range(100)]
filesets = {
    'train': all_files[:70],
    'val': all_files[70:80],
    'test': all_files[80:]
    }

In [ ]:
def get_data(fileset, group):
    ds = LineDataFlow(fileset, do_shuffle=group == 'train')
    ds = tp.BatchData(ds, BATCH)
    return ds

In [ ]:
# base feature count width
NF = 16

INITS = {
    'TNC': tf.truncated_normal_initializer(stddev=0.02),
    'VSI': tf.contrib.layers.variance_scaling_initializer(factor=1.0, mode='FAN_AVG', uniform=True)
}

def visualize_conv_activations(activation, name):
    """Visualize activations for convolution layers.

    Remarks:
        This tries to place all activations into a square.

    Args:
        activation: tensor with the activation [B,H,W,C]
        name: label for tensorboard

    Returns:
        image of almost all activations
    """
    import math
    with tf.name_scope('visualize_act_' + name):
        _, h, w, c = activation.get_shape().as_list()
        rows = []
        c_per_row = int(math.sqrt(c))
        for y in range(0, c - c_per_row, c_per_row):
            row = activation[:, :, :, y:y + c_per_row]  # [?, H, W, 32] --> [?, H, W, 5]
            cols = tf.unstack(row, axis=3)              # [?, H, W, 5] --> 5 * [?, H, W]
            row = tf.concat(cols, 1)
            rows.append(row)

        viz = tf.concat(rows, 2)
    tf.summary.image('visualize_act_' + name, tf.expand_dims(viz, -1))

def coord_to_points(arr, coords, coords_, idy, idx):
    # coord quadruplets only
    arr_B = arr * 255.0
    # Blue = mask
    
    with tf.name_scope('viz_points'):
        # Label coords (bigger) - green
        arr_G = tf.zeros_like(arr)
        for coord in tf.split(coords, 4, axis=1): # -> (B, yx)
            coord = tf.reshape(coord, [-1, 1, 1, 2])
            dysq = tf.square(idy - coord[..., 0] * DIMS[0])
            dxsq = tf.square(idx - coord[..., 1] * DIMS[1])
            raster = tf.less_equal(dysq + dxsq, 2.0)
            raster = tf.expand_dims(raster, axis=3)
            arr_G += tf.cast(raster, tf.float32)
            
        # Guessed coords (smaller) - red
        arr_R = tf.zeros_like(arr)
        for coord in tf.split(coords_, 4, axis=1): # -> (B, yx)
            coord = tf.reshape(coord, [-1, 1, 1, 2])
            dysq = tf.square(idy - coord[..., 0] * DIMS[0])
            dxsq = tf.square(idx - coord[..., 1] * DIMS[1])
            raster = tf.less_equal(dysq + dxsq, 1.5)
            raster = tf.expand_dims(raster, axis=3)
            arr_R += tf.cast(raster, tf.float32)
    
        return tf.concat([arr_R * 255, arr_G * 255, arr_B], axis=3)

class Model(tp.ModelDesc):
    
    def _get_inputs(self):
        return [tp.InputDesc(tf.float32, (None,) + DIMS, 'mask'),
                tp.InputDesc(tf.float32, (None, 4, 2), 'param'),
                tp.InputDesc(tf.float32, (None, 1), 'width')
                ]

    def networkVGG(self, x, idy, idx):
        """
        VGG Net. Baseline A.
        """
        with tp.argscope(tp.Conv2D, kernel_shape=3, stride=1, nl=tf.nn.elu):
            c0 = tp.Conv2D('conv0', x, NF)
            
            if USE_IDYX:
                l = tf.concat([c0 * idy / DIMS[0], c0 * idx / DIMS[1]], axis=3)
            
            p0 = tp.MaxPooling('pool0', c0, 2) # -> 32x
            c1 = tp.Conv2D('conv1', p0, NF * 2)
            c2 = tp.Conv2D('conv2', c1, NF * 4)
            p1 = tp.MaxPooling('pool1', c2, 2) # -> 16x
            c3 = tp.Conv2D('conv3', p1, NF * 4)
            c4 = tp.Conv2D('conv4', c3, NF * 8)
            p2 = tp.MaxPooling('pool2', c4, 2) # -> 8x
            f1 = tp.FullyConnected('fc0', p2, 512)
            l  = tp.Dropout('drop', f1, 0.5)
            
            # Eh, dims are symmetric anyway
            param = tp.FullyConnected('fc_param', l, 8, nl=tf.identity)
            param = tf.reshape(param, [-1, 4, 2])
            
            width = tp.FullyConnected('fc_width', l, 1, nl=tf.identity)

            visualize_conv_activations(c0, 'c0')
            visualize_conv_activations(c2, 'c2')
            visualize_conv_activations(c4, 'c4')
            
        return param, width
    
    def networkSeq1(self, x, idy, idx):
        """
        Sequential network with factorized filters.
        Get outer points first, then inner points.
        """
        with tp.argscope(tp.Conv2D, kernel_shape=3, stride=1, nl=tf.nn.elu):
            # Broad filter across first
            c0y = tp.Conv2D('conv0y',   x, NF, kernel_shape=[7, 1])
            c0x = tp.Conv2D('conv0x', c0y, NF, kernel_shape=[1, 7])
            c1  = tp.Conv2D('conv1',  c0x, NF, kernel_shape=[3, 3])
            
            if USE_IDYX:
                yx = tf.concat([c1, c1 * idy / DIMS[0], c1 * idx / DIMS[1]], axis=3)
            
            # Outer control points' heatmap
            c_outer = tp.Conv2D('conv2o', yx, 4)
            outer = tp.GlobalAvgPooling('gap_outer', c_outer) # -> 4
            outer = tf.reshape(outer, [-1, 2, 2])
            point_1, point_4 = tf.split(outer, 2, axis=1)

            # Width
            c_width = tp.Conv2D('conv2w', c1, 4)
            width = tp.FullyConnected('fc_width', c_width, 1, nl=tf.identity)

            # Use outer point heatmap and yx to discover inner points
            remix = tf.concat([c_outer, yx], axis=3)
            c2 = tp.Conv2D('conv2i', remix, NF * 2)
            c3 = tp.Conv2D('conv3', c2, NF * 4)
            p2 = tp.AvgPooling('pool2', c3, 2) # -> 32x
            c_inner = tp.Conv2D('conv4', p2, NF * 4)
            g_inner = tp.GlobalAvgPooling('gap_inner', c_inner) # -> 1 x 4NF
            inner = tp.FullyConnected('fc_param', g_inner, 4, nl=tf.identity)
            inner = tf.reshape(inner, [-1, 2, 2])
            point_2, point_3 = tf.split(inner, 2, axis=1)

            param = tf.concat([point_1, point_2, point_3, point_4], axis=1)

            visualize_conv_activations(yx, 'yx')
            visualize_conv_activations(c2, 'c2')
            visualize_conv_activations(c_outer, 'c_outer')
            visualize_conv_activations(c_inner, 'c_inner')
            visualize_conv_activations(c_width, 'c_width')
            
        return param, width
    
    def networkSeq2(self, x, idy, idx):
        """
        Sequential network with factorized filters.
        Get outer points first, then inner points.
        """
        with tp.argscope(tp.Conv2D, kernel_shape=3, stride=1, nl=tf.nn.elu):
            # Broad filter across first
            c0y = tp.Conv2D('conv0y',   x, NF, kernel_shape=[7, 1])
            c0x = tp.Conv2D('conv0x', c0y, NF, kernel_shape=[1, 7])
            c1  = tp.Conv2D('conv1',  c0x, NF, kernel_shape=[5, 5])
            cw  = tp.Conv2D('conv1b',  c1, NF, kernel_shape=[5, 5], nl=tf.sigmoid)
            
            if USE_IDYX:
                yx = tf.concat([c1, cw * idy / DIMS[0], cw * idx / DIMS[1]], axis=3)
            
            # Outer control points' heatmap
            l       = tp.Conv2D('conv2o0', yx, 4 * NF)
            l       = tp.Conv2D('conv2o1',  l, 4 * NF)
            c_outer = tp.Conv2D('conv2o2',  l, 4, nl=tf.sigmoid)
            outer = tp.GlobalAvgPooling('gap_outer', c_outer) # -> 4
            outer = tf.reshape(outer, [-1, 2, 2])

            # Width
            c_width = tp.Conv2D('conv2w', c1, 4)
            width = tp.FullyConnected('fc_width', c_width, 1, nl=tf.identity)

            # Use outer point heatmap and yx to discover inner points
            remix = tf.concat([c_outer, yx], axis=3)
            c2 = tp.Conv2D('conv2i', remix, NF * 2)
            c3 = tp.Conv2D('conv3', c2, NF * 4)
            p2 = tp.AvgPooling('pool2', c3, 2) # -> 32x
            c4 = tp.Conv2D('conv4', p2, NF * 4)
            c_inner = tp.Conv2D('conv5', c4, NF * 4, nl=tf.sigmoid)
            g_inner = tp.GlobalAvgPooling('gap_inner', c_inner) # -> 1 x 4NF
            inner = tp.FullyConnected('fc_param', g_inner, 4, nl=tf.identity)
            inner = tf.reshape(inner, [-1, 2, 2])

            point_1, point_4 = tf.split(outer, 2, axis=1)
            point_2, point_3 = tf.split(inner, 2, axis=1)
            param = tf.concat([point_1, point_2, point_3, point_4], axis=1)

            visualize_conv_activations(cw, 'yx')
            visualize_conv_activations(yx, 'yx')
            visualize_conv_activations(c2, 'c2')
            visualize_conv_activations(c_outer, 'c_outer')
            visualize_conv_activations(c_inner, 'c_inner')
            visualize_conv_activations(c_width, 'c_width')
            
        return param, width

    def networkFWB(self, x, idy, idx):
        """
        Normal deep feedforward with combinations of indexing and mask in beginning.
        """
        with tp.argscope(tp.Conv2D, kernel_shape=3, stride=1, nl=tf.nn.elu):

            # +/- mask * idyx
            cyp =  x * idy / DIMS[0]
            cym = -x * idy / DIMS[0]
            cxp =  x * idx / DIMS[1]
            cxm = -x * idx / DIMS[1]
            l = tf.concat([cxp, cxm, cyp, cym], axis=3)
            c0 = tp.Conv2D('conv0', l, NF * 2, kernel_shape=7)
            p0 = tp.AvgPooling('pool0', l, 2) # -> 32x
            p0  = tp.Dropout('drop1', p0, 0.9)
            c1 = tp.Conv2D('conv1', p0, NF * 2)
            c2 = tp.Conv2D('conv2', c1, NF * 4)
            p1 = tp.AvgPooling('pool1', c2, 2) # -> 16x
            p1  = tp.Dropout('drop1', p1, 0.8)
            c3 = tp.Conv2D('conv3', p1, NF * 4)
            c4 = tp.Conv2D('conv4', c3, NF * 8)
            p2 = tp.AvgPooling('pool2', c4, 2) # -> 8x
            p2  = tp.Dropout('drop2', p2, 0.7)
            c5 = tp.Conv2D('conv5', p2, NF * 8)
            c6 = tp.Conv2D('conv6', c5, NF * 16)
            p3 = tp.AvgPooling('pool2', c6, 2) # -> 4x
            f1 = tp.FullyConnected('fc0', p3, 256)
            l  = tp.Dropout('dropf', f1, 0.5)

            param = tp.FullyConnected('fc_param', l, 8, nl=tf.sigmoid) * 1.1
            param = tf.reshape(param, [-1, 4, 2]) - 0.05
            
            width = tp.FullyConnected('fc_width', l, 1, nl=tf.sigmoid) * 2.0

            visualize_conv_activations(c0, 'c0')
            visualize_conv_activations(c2, 'c2')
            visualize_conv_activations(c4, 'c4')
            visualize_conv_activations(c6, 'c6')
            
        return param, width
    
    def _build_graph(self, inputs):
        mask, param, width = inputs
        
        with tf.name_scope('indexing'):
            np_idx = np.c_[:DIMS[0]] + np.zeros((1, DIMS[0]))
            np_idx = np_idx[None, ...]
            np_idy = np.r_[:DIMS[1]] + np.zeros((DIMS[0], 1))
            np_idy = np_idy[None, ...]
            idy = tf.convert_to_tensor(np_idy, dtype='float32')
            idx = tf.convert_to_tensor(np_idx, dtype='float32')
        
        # Standardize
        with tf.name_scope('prepare_input'):
            mask = tf.expand_dims(mask, axis=3)
            x = 0.5 - mask
            
            param = param / DIMS[0]
        
        net_options = {
            'vgg': self.networkVGG,
            'seq1': self.networkSeq1,
            'seq2': self.networkSeq2,
            'fwb': self.networkFWB
        }
        with tp.argscope([tp.Conv2D], W_init=INITS['VSI']):
            with tf.variable_scope('encoder'):
                param_, width_ = net_options[NET_OPTION](x,
                                                         tf.expand_dims(idy, axis=3),
                                                         tf.expand_dims(idx, axis=3))
        
        with tf.name_scope('outputs'):
            param_ = tf.identity(param_, name='params')
            width_ = tf.identity(width_, name='widths')
            
        with tf.name_scope('loss'):
            # MSE losses (reduce to 0-1)
            p_loss = tf.reduce_mean(tf.square(param_ - param), axis=(1, 2))
            tf.summary.histogram(values=p_loss, name='summary-params')
            p_loss = tf.reduce_mean(p_loss, name='params')
            
            w_loss = tf.reduce_mean(tf.square(width_ - width), axis=(1))
            tf.summary.histogram(values=w_loss, name='summary-widths')
            w_loss = tf.reduce_mean(w_loss, name='widths')
            
            add_moving_summary(p_loss, w_loss)
            cost = p_loss + w_loss
            
            reg_loss = tf.multiply(1e-5, tp.regularize_cost('fc.*/W', tf.nn.l2_loss),
                                   name='regularize_loss')
            self.cost = tf.add_n([reg_loss, cost], name='total_cost')
            add_moving_summary(self.cost)
            
        # Expose to circumvent naming / folder issue
        p_loss = tf.identity(p_loss, name='loss_params')
        
        if tp.tfutils.get_current_tower_context().is_training:
            tf.summary.image('mask', mask, max_outputs=10)
            tf.summary.image('points', coord_to_points(mask, param, param_, idx, idy), max_outputs=10)

    def _get_optimizer(self):
        lr = symbf.get_scalar_var('learning_rate', 4e-4, summary=True)
        return tf.train.AdamOptimizer(lr, beta1=0.5, epsilon=1e-3)

In [ ]:
LOAD = './train_log/linenetFwbVSI3/model-3010'

dataflows = {group: get_data(fileset, group) for group, fileset in filesets.items()}

tp.logger.set_logger_dir('./train_log/linenetFwbVSI3')

In [ ]:
tf.reset_default_graph()
config = tp.TrainConfig(
    model=Model(),
    dataflow=dataflows['train'],  # the DataFlow instance for training
    callbacks=[
        tp.ModelSaver(),
        tp.MinSaver('loss_params'),
        tp.InferenceRunner(
            dataflows['val'],
            # Calculate both the cost and the error for this DataFlow
            [tp.ScalarStats('loss/params'),
             tp.ScalarStats('loss/widths')]),
    ],
    steps_per_epoch=dataflows['train'].size(),
    max_epoch=1000,
)

if LOAD:
    config.session_init = tp.SaverRestore(LOAD)

tp.QueueInputTrainer(config).train()

In [ ]:
for i in dataflows['train'].get_data():
    q = i
    break
qm, qp, qw = q
import matplotlib.pyplot as plt
plt.imshow(qm[2], cmap=plt.cm.gray_r)
plt.plot(qp[2, :, 1], qp[2, :, 0], color='red')
plt.show()

tp.

In [ ]:








testconfig = tp.PredictConfig(
                  session_init=tp.get_model_loader('./train_log/linenetFwbVSI2/model-16870'),
                  model=Model(),
                  input_names=['mask'],
                  output_names=['outputs/params', 'outputs/widths'])

predictor = tp.OfflinePredictor(testconfig)






In [ ]:
# predictor()

for datapoint in dataflows['test'].get_data():
    predictor(datapoint)